In [88]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

# Import API key
from config import gkey

In [89]:
# Import College Football Stadiums file into the stadiums_pd DataFrame
stadiums_pd = pd.read_csv("Resources/College_Football_Stadiums.csv")

stadiums_pd

stadiums_pd = stadiums_pd.drop(columns="Image")

stadiums_pd

,Stadium,City,State,Team,Conference,Capacity,Record,Built,Expanded 2,Surface
0,Aggie Memorial Stadium,Las Cruces,NM,New Mexico State,Independent,"30,343","32,993(September 26, 1998 vs. UTEP)[1]",1978,2005[1],S5-M Synthetic Turf
1,Alamodome,San Antonio,TX,UTSA,C-USA,"65,000","56,743(September 3, 2011 vs. Northeastern State)",1993,NaN,SportField
2,Alaska Airlines Field at Husky Stadium,Seattle,WA,Washington,Pac-12,"70,083","76,125(September 23, 1995 vs. Army)",1920,2013,FieldTurf
3,Albertsons Stadium,Boise,ID,Boise State,Mountain West,"36,387","36,864(September 20, 2012 vs. BYU)[4]",1970,2012[3],Smurf Turf
4,Allegiant Stadium,Paradise,NV,UNLV,Mountain West,"65,000",NaN,2020,NaN,FieldTurf
...,...,...,...,...,...,...,...,...,...,...
125,Veterans Memorial Stadium at Larry Blakeney Field,Troy,AL,Troy,Sun Belt,"30,402","29,013(September 15, 2012 vs. Mississippi State)",1950,2003 & 2018,ProGrass
126,Waldo Stadium,Kalamazoo,MI,Western Michigan,MAC,"30,200","36,361(September 16, 2000 vs. Indiana State)",1939,2003[153],FieldTurf
127,Warren McGuirk Alumni Stadium,Hadley,MA,UMass,Independent,"17,000","20,000(November 25, 1972 vs. Boston College)",1965,2014,FieldTurf
128,Wayne Day Family Field at Carter–Finley Stadium,Raleigh,NC,NC State,ACC,"57,583","57,583(multiple times)",1966,2006,Tifway 419 Bermuda Grass


In [90]:
# Add columns for lat, lng
# Hint: Be sure to specify "" as the initial value for each column.

stadiums_ordered_pd = stadiums_pd[["Stadium","City","State","Team","Conference","Capacity","Record","Built","Surface"]]

stadiums_ordered_pd.head()

stadiums_ordered_pd.head()

,Stadium,City,State,Team,Conference,Capacity,Record,Built,Surface
0,Aggie Memorial Stadium,Las Cruces,NM,New Mexico State,Independent,"30,343","32,993(September 26, 1998 vs. UTEP)[1]",1978,S5-M Synthetic Turf
1,Alamodome,San Antonio,TX,UTSA,C-USA,"65,000","56,743(September 3, 2011 vs. Northeastern State)",1993,SportField
2,Alaska Airlines Field at Husky Stadium,Seattle,WA,Washington,Pac-12,"70,083","76,125(September 23, 1995 vs. Army)",1920,FieldTurf
3,Albertsons Stadium,Boise,ID,Boise State,Mountain West,"36,387","36,864(September 20, 2012 vs. BYU)[4]",1970,Smurf Turf
4,Allegiant Stadium,Paradise,NV,UNLV,Mountain West,"65,000",NaN,2020,FieldTurf


In [91]:
from pprint import pprint

for index, row in stadiums_ordered_pd.iterrows():
    
    target_city = row['Stadium']
    
    base_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={target_city}&key={gkey}"
    
    #lat = row['Lat']
    #lng = row['Lng']

    # add keyword to params dict
    params['location'] = f'{lat},{lng}'
    
    # assemble 

    response = requests.get(base_url, params=params).json() 
    
    try:
        stadiums_ordered_pd.loc[index, 'Latitude'] = response['results'][0]['geometry']['location']['lat']
        stadiums_ordered_pd.loc[index, 'Longitude'] = response['results'][0]['geometry']['location']['lng'] 
    
    except:
        print(target_city)
    
stadiums_ordered_pd

,Stadium,City,State,Team,Conference,Capacity,Record,Built,Surface,Latitude,Longitude
0,Aggie Memorial Stadium,Las Cruces,NM,New Mexico State,Independent,"30,343","32,993(September 26, 1998 vs. UTEP)[1]",1978,S5-M Synthetic Turf,32.279486,-106.741713
1,Alamodome,San Antonio,TX,UTSA,C-USA,"65,000","56,743(September 3, 2011 vs. Northeastern State)",1993,SportField,29.416983,-98.478814
2,Alaska Airlines Field at Husky Stadium,Seattle,WA,Washington,Pac-12,"70,083","76,125(September 23, 1995 vs. Army)",1920,FieldTurf,47.650350,-122.301611
3,Albertsons Stadium,Boise,ID,Boise State,Mountain West,"36,387","36,864(September 20, 2012 vs. BYU)[4]",1970,Smurf Turf,43.602844,-116.195866
4,Allegiant Stadium,Paradise,NV,UNLV,Mountain West,"65,000",NaN,2020,FieldTurf,36.090866,-115.183322
...,...,...,...,...,...,...,...,...,...,...,...
125,Veterans Memorial Stadium at Larry Blakeney Field,Troy,AL,Troy,Sun Belt,"30,402","29,013(September 15, 2012 vs. Mississippi State)",1950,ProGrass,31.799556,-85.951833
126,Waldo Stadium,Kalamazoo,MI,Western Michigan,MAC,"30,200","36,361(September 16, 2000 vs. Indiana State)",1939,FieldTurf,42.285756,-85.601090
127,Warren McGuirk Alumni Stadium,Hadley,MA,UMass,Independent,"17,000","20,000(November 25, 1972 vs. Boston College)",1965,FieldTurf,42.377271,-72.536040
128,Wayne Day Family Field at Carter–Finley Stadium,Raleigh,NC,NC State,ACC,"57,583","57,583(multiple times)",1966,Tifway 419 Bermuda Grass,35.800800,-78.719566


In [92]:
stadiums_cleaned_df = stadiums_ordered_pd[["Stadium", "Latitude", "Longitude", "City", "State", "Team", "Conference", "Capacity", "Record", "Built", "Surface"]]

stadiums_cleaned_df

,Stadium,Latitude,Longitude,City,State,Team,Conference,Capacity,Record,Built,Surface
0,Aggie Memorial Stadium,32.279486,-106.741713,Las Cruces,NM,New Mexico State,Independent,"30,343","32,993(September 26, 1998 vs. UTEP)[1]",1978,S5-M Synthetic Turf
1,Alamodome,29.416983,-98.478814,San Antonio,TX,UTSA,C-USA,"65,000","56,743(September 3, 2011 vs. Northeastern State)",1993,SportField
2,Alaska Airlines Field at Husky Stadium,47.650350,-122.301611,Seattle,WA,Washington,Pac-12,"70,083","76,125(September 23, 1995 vs. Army)",1920,FieldTurf
3,Albertsons Stadium,43.602844,-116.195866,Boise,ID,Boise State,Mountain West,"36,387","36,864(September 20, 2012 vs. BYU)[4]",1970,Smurf Turf
4,Allegiant Stadium,36.090866,-115.183322,Paradise,NV,UNLV,Mountain West,"65,000",NaN,2020,FieldTurf
...,...,...,...,...,...,...,...,...,...,...,...
125,Veterans Memorial Stadium at Larry Blakeney Field,31.799556,-85.951833,Troy,AL,Troy,Sun Belt,"30,402","29,013(September 15, 2012 vs. Mississippi State)",1950,ProGrass
126,Waldo Stadium,42.285756,-85.601090,Kalamazoo,MI,Western Michigan,MAC,"30,200","36,361(September 16, 2000 vs. Indiana State)",1939,FieldTurf
127,Warren McGuirk Alumni Stadium,42.377271,-72.536040,Hadley,MA,UMass,Independent,"17,000","20,000(November 25, 1972 vs. Boston College)",1965,FieldTurf
128,Wayne Day Family Field at Carter–Finley Stadium,35.800800,-78.719566,Raleigh,NC,NC State,ACC,"57,583","57,583(multiple times)",1966,Tifway 419 Bermuda Grass


In [95]:
stadiums_cleaned_df.to_csv (r'C:\Users\calicritic93\College Football Stadiums (Project 3)\stadiums_cleaned.csv', index = False, header=True)